In [8]:
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from uszipcode import SearchEngine

import utils.lifesense_utils as ls_utils

from multiprocessing import Pool

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

%matplotlib inline

In [33]:
ema_file_prefix = "/data/tliu/wave2_aggregate/2020-05-04/"
vad_file_loc = "/data/tliu/wave2_vad/2020-02-01_2020-04-01/pdk-voice-activity/{}.df"

# VAD label processing

In [3]:
pids = []
wave2_id_file_loc = "/home/tliu/lifesense/wave2_data/wave2_wk4_ids.txt"
with open(wave2_id_file_loc, "r") as wave_f:
    for line in wave_f.readlines():
        pids.append(line.strip())

In [4]:
phq_file_loc = ema_file_prefix + "{}_phq8/{}.df"

time_of_days = ['morning', 'evening']

phq_df = pd.DataFrame()
for pid in pids:
    for tod in time_of_days:
        df = pd.read_pickle(phq_file_loc.format(tod, pid))
        if df.shape[0] > 0:
            #df = df[keep_cols]
            #df['time_of_day'] = tod
            phq_df = phq_df.append(df, sort=True)
            
    break

In [9]:
phq_df = ls_utils.format_time(phq_df)

In [15]:
vad_cols = phq_df.columns[phq_df.columns.str.contains("voice")].tolist()
phq_df[['timestamp', 'date'] + vad_cols]

,timestamp,date,voice_relationship_1581903306645_1581908406759,voice_relationship_1581912006794_1581912906805,voice_relationship_1581915006833_1581915606838,voice_relationship_1581919208561_1581919808567,voice_relationship_1581932716373_1581933319359,voice_relationship_1583726408001_1583727631000,voice_relationship_1583733941976_1583735141988,voice_relationship_1583778361920_1583779563516,...,voice_relationship_other_1583726408001_1583727631000,voice_relationship_other_1583733941976_1583735141988,voice_relationship_other_1583778361920_1583779563516,voice_relationship_other_1583786466574_1583787366600,voice_relationship_other_1583791078199_1583792578514,voice_relationship_other_1585539499369_1585540399405,voice_relationship_other_1585548799558_1585549399566,voice_relationship_other_1585592305819_1585592905824,voice_relationship_other_1585610610988_1585611510997,voice_relationship_other_1585613311037_1585614211048
0,1583260211,2020-03-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1585069818,2020-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1586891881,2020-04-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,1581993799,2020-02-17,[relationship_acquaintance],[relationship_acquaintance],[relationship_family_live_away],[relationship_friend],[relationship_friend],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1583798974,2020-03-09,NaN,NaN,NaN,NaN,NaN,[relationship_none],[relationship_none],[relationship_none],...,,,,,,NaN,NaN,NaN,NaN,NaN
2,1585615377,2020-03-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,,,,,


In [18]:
ema_file_loc = ema_file_prefix + "{}_ema/{}.df"

time_of_days = ['evening']

ema_df = pd.DataFrame()
for pid in pids:
    for tod in time_of_days:
        df = pd.read_pickle(ema_file_loc.format(tod, pid))
        if df.shape[0] > 0:
            #df = df[keep_cols]
            #df['time_of_day'] = tod
            ema_df = ema_df.append(df, sort=True)
            
    break

In [19]:
ema_df.columns

Index(['cluster_distance', 'cluster_min_points',
       'contact_close_0102898cd094229be3d131d14b9f5801269d52bee0d1a4a629f45ac6a33b6ea4',
       'contact_close_02d7a36ab672761399585a1ef6dccc54537a99afc78c339ad3c06a885111b139',
       'contact_close_0b7683f0ae1b2bca5a14ad8d1f09ed76fd82362b9c297ca96ad6a76dddd6b222',
       'contact_close_0fe8ef5169b50745410c984241b2eda4ffe5a529867745c5d267f284528c89b3',
       'contact_close_2406385dc4921c522a13b24f60d1f018322cea173394ef09688917f23501bbb9',
       'contact_close_44d2e6e594a6707336bd5a66cd57de7d557de441546495a23083ac36e62b5d46',
       'contact_close_57b67ec92806ed0b71d7156d4a984ad96d062e3892d8451362b2dfd2599e9603',
       'contact_close_5b18230e1cc9b8e8ede276fae86bbc6d8e6840a6afcef76357dbc72385375fc8',
       ...
       'voice_relationship_other_1585449555634_1585450155644',
       'voice_relationship_other_1585451955658_1585454955698',
       'voice_relationship_other_1585455555704_1585456155721',
       'voice_relationship_other_158545

In [23]:
voice_cols = ema_df.columns[ema_df.columns.str.contains('voice')].tolist()
voice_cols

['voice_relationship_1581520682062_1581523384996',
 'voice_relationship_1581540356655_1581541258779',
 'voice_relationship_1581542758790_1581543358817',
 'voice_relationship_1581547859770_1581549059780',
 'voice_relationship_1581549659789_1581551161840',
 'voice_relationship_1581664220688_1581664820692',
 'voice_relationship_1581705631156_1581706231165',
 'voice_relationship_1581708631203_1581709531224',
 'voice_relationship_1581723632222_1581724532238',
 'voice_relationship_1581724940990_1581728241022',
 'voice_relationship_1581730341041_1581732441057',
 'voice_relationship_1581733042201_1581734243245',
 'voice_relationship_1581807646860_1581808546867',
 'voice_relationship_1581809746876_1581810346882',
 'voice_relationship_1581813646924_1581814248473',
 'voice_relationship_1581815748491_1581817248523',
 'voice_relationship_1581819048562_1581819948595',
 'voice_relationship_1581820848609_1581822348655',
 'voice_relationship_1581826250088_1581826850094',
 'voice_relationship_1581831600

In [28]:
ema_df['voice_relationship_1583452335649_1583452935663']

0                               NaN
1                               NaN
2                               NaN
3                               NaN
4     [relationship_spouse_partner]
5                               NaN
6                               NaN
7                               NaN
8                               NaN
9                               NaN
10                              NaN
11                              NaN
12                              NaN
13                              NaN
Name: voice_relationship_1583452335649_1583452935663, dtype: object

## Extract voice relationship columns

In [74]:
def extract_voice_df(ema_df, pid):
    """Extracts voice relationship data from EMA dataframes.
    
    Args:
        pid (str): the participant id
        ema_df (pd.DataFrame): an evening_ema or evening_phq8 survey DataFrame
    
    Returns:
        voice_df (pd.DataFrame):
            pid (str)
            relationship (str)
            start_timestamp (int)
            start_time (pd.DateTime)
            end_timestamp (int)
            end_time (pd.DateTime)
            duration (pd.TimeDelta)
    """

    voice_cols = ema_df.columns[ema_df.columns.str.contains('voice')].tolist()
    df_dict = {
        "relationship": [],
        "start_timestamp": [],
        "start_time": [],
        "end_timestamp": [],
        "end_time": [],
        "duration": []
    }
    for col in voice_cols:

        # extract relationship string
        relation_str = ""
        if "other" in col:
            relation_str += "other"
        rel_val = ema_df[col][~ema_df[col].isna()].iloc[0].strip("[]")
        rel_split = rel_val.split('_')
        if rel_split[0] == "relationship":
            rel_split.pop(0)
        relation_str += "_".join(rel_split)

        # strip any leading or trailing whitespace
        relation_str = relation_str.strip()    


        # extract start and end times: last two elements of the col
        col_split = col.split("_")
        start_time = int(col_split[-2])
        end_time = int(col_split[-1])

        # populate df_dict
        df_dict['relationship'].append(relation_str)
        df_dict['start_timestamp'].append(start_time)
        df_dict['start_time'].append(pd.to_datetime(start_time, unit="ms"))
        df_dict['end_timestamp'].append(end_time)
        df_dict['end_time'].append(pd.to_datetime(end_time, unit="ms"))
        df_dict['duration'].append(pd.to_timedelta(end_time - start_time, unit="ms"))

    voice_df = pd.DataFrame.from_dict(df_dict)
    voice_df['pid'] = pid
    return voice_df

### Evening PHQ extraction

In [73]:
voice_df = pd.DataFrame()

In [75]:
phq_file_loc = ema_file_prefix + "{}_phq8/{}.df"

time_of_days = ['morning', 'evening']

for pid in pids:
    for tod in time_of_days:
        df = pd.read_pickle(phq_file_loc.format(tod, pid))
        if df.shape[0] > 0:
            v_df = extract_voice_df(df, pid)
            voice_df = voice_df.append(v_df, sort=True)

### Evening EMA extraction

In [76]:
ema_file_loc = ema_file_prefix + "{}_ema/{}.df"

time_of_days = ['evening']

for pid in pids:
    for tod in time_of_days:
        df = pd.read_pickle(ema_file_loc.format(tod, pid))
        if df.shape[0] > 0:
            v_df = extract_voice_df(df, pid)
            voice_df = voice_df.append(v_df, sort=True)

In [77]:
voice_df.shape

(21486, 7)

In [78]:
voice_df.head()

,duration,end_time,end_timestamp,pid,relationship,start_time,start_timestamp
0,01:25:00.114000,2020-02-17 03:00:06.759,1.581908e+12,14673820,acquaintance,2020-02-17 01:35:06.645,1.581903e+12
1,00:15:00.011000,2020-02-17 04:15:06.805,1.581913e+12,14673820,acquaintance,2020-02-17 04:00:06.794,1.581912e+12
2,00:10:00.005000,2020-02-17 05:00:06.838,1.581916e+12,14673820,family_live_away,2020-02-17 04:50:06.833,1.581915e+12
3,00:10:00.006000,2020-02-17 06:10:08.567,1.581920e+12,14673820,friend,2020-02-17 06:00:08.561,1.581919e+12
4,00:10:02.986000,2020-02-17 09:55:19.359,1.581933e+12,14673820,friend,2020-02-17 09:45:16.373,1.581933e+12


# VAD Dataframe

In [34]:
test_pid = "01515998"

vad_df = pd.read_pickle(vad_file_loc.format(test_pid))

In [37]:
vad_df['voices_present'].value_counts()

True     13073
False     5300
Name: voices_present, dtype: int64

In [39]:
vad_df[['level', 'observed', 'power', 'smooth_mode', 'smooth_window']]

,level,observed,power,smooth_mode,smooth_window
0,1.0,1580931985508,NaN,median,13
1,0.8,1580932045518,NaN,median,13
2,0.8,1580932105532,NaN,median,13
3,0.8,1580932165537,NaN,median,13
4,1.0,1580932225541,NaN,median,13
5,0.8,1580932285552,NaN,median,13
6,0.6,1580932345559,NaN,median,13
7,0.6,1580932405572,NaN,median,13
8,1.0,1580932465583,NaN,median,13
9,0.8,1580932525594,NaN,median,13


# Notes

## voice relationship dataframe

- want to convert ema and phq into "voice_relationship" dataframe:
    - three columns: relationship, start time, end time, conversation length

## VAD dataframe